### Save and Retrive Data from Redis
- https://redis.com/

In [22]:
import numpy as np
import pandas as pd
import redis

In [23]:
# Connect to Redis Client
# redis-18692.c57.us-east-1-4.ec2.cloud.redislabs.com:18692
# wCneC8tYJRuw0ovU9Kacbr9kgY4UmQdb
hostname = 'redis-15185.c305.ap-south-1-1.ec2.redns.redis-cloud.com'
portnumber = 15185
password = 'vfJ9zVbFFbzPueK4ttiVuANMwsQ0VPrd'

r = redis.StrictRedis(host=hostname,
                      port=portnumber,
                      password=password)


In [24]:
r.ping()

True

In [25]:
#r.keys()

#### Load the data (numpy zip)

In [26]:
np_file = np.load('dataframe_students_teacher.npz',allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values,columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.43240861892700194, 1.0746600151062011, -0.1..."
1,Barack Obama,Teacher,"[0.9487079620361328, -0.558357048034668, 0.094..."
2,Chris Evans,Student,"[0.7200980663299561, -1.468521785736084, -0.77..."
3,Morgan Freeman,Teacher,"[0.48542585372924807, -0.6193629741668701, -0...."
4,Scarlett Johansson,Student,"[0.1913154363632202, -0.48847031593322754, -1...."


In [27]:
df['name_role'] = df['Name']+'@'+df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.43240861892700194, 1.0746600151062011, -0.1...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.9487079620361328, -0.558357048034668, 0.094...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.7200980663299561, -1.468521785736084, -0.77...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.48542585372924807, -0.6193629741668701, -0....",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.1913154363632202, -0.48847031593322754, -1....",Scarlett Johansson@Student


In [28]:
records = df[['name_role','Facial_Features']].to_dict(orient='records')
len(records)

5

## Save data in Redis
- redis hashes
- keyname = 'academy:register'

In [29]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']
    
    # convert numpy array into bytes
    vector_bytes = vector.tobytes()
    
    # save data into redis cloud
    r.hset(name='academy:register',key=name_role,value=vector_bytes)

In [30]:
vector.dtype

dtype('float64')

### Retrive Records from Redis

In [31]:
retrive_dict = r.hgetall(name='academy:register')
retrive_series = pd.Series(retrive_dict)
retrive_series

b'Angelina Jolie@Student'        b'3333\x95\xac\xdb?333\xb3\xce1\xf1?3333xx\xc0...
b'Chris Evans@Student'           b'\x9a\x99\x99\x19\x0b\x0b\xe7?333\xb3\x10\x7f...
b'Aditya@Student'                b'\x88\xcb4>\x036r?\x1c\x1dR?J\x02\x89?\x91\xe...
b'Barack Obama@Teacher'          b'\xcd\xcc\xcc\xcc\xd0[\xee?\x9a\x99\x99\x99\x...
b'Scarlett Johansson@Student'    b'3333\x06}\xc8?\x00\x00\x00\x00\x19C\xdf\xbf\...
b'Ranbir@Student'                b'O\xdc\x8f?;\xdc\x1a?\xc0\xc4!\xbf\xcfD\x15\x...
b'Sharukh Khan@Teacher'          b'\xc1w\xa6?\x88\x99h>\x9a]\x15>\x03\xb7\xb5\x...
b'Salman @Student'               b'\xda\xedu\xbe\xa3\xfat\xbf\x85\xc7\x88\xbf\x...
b'Morgan Freeman@Teacher'        b'\x9a\x99\x99\x997\x11\xdf?\xcd\xcc\xccL\xd2\...
dtype: object

In [32]:
#np.frombuffer(retrive_series[b'Morgan Freeman@Teacher'],dtype=np.float32).shape
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))

In [33]:
retrive_series.index = index

In [34]:
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df

,name_role,facial_features
0,Angelina Jolie@Student,"[4.172325e-08, 1.7162043, -4.172325e-08, 1.884..."
1,Chris Evans@Student,"[1.5881868e-23, 1.8050245, -4.172325e-08, -1.9..."
2,Aditya@Student,"[0.17655766, 0.94613665, 0.8207567, 1.0703824,..."
3,Barack Obama@Teacher,"[-107374184.0, 1.8621769, -1.5881868e-23, -1.7..."
4,Scarlett Johansson@Student,"[4.172325e-08, 1.5663154, 0.0, -1.7442352, -1...."
5,Ranbir@Student,"[1.1239108, 0.60492295, -0.6319084, -0.5830812..."
6,Sharukh Khan@Teacher,"[1.3005296, 0.22714818, 0.14586487, -0.3549119..."
7,Salman @Student,"[-0.24016514, -0.9569494, -1.0685889, 0.797408..."
8,Morgan Freeman@Teacher,"[-1.5881868e-23, 1.7427129, 107374184.0, -1.77..."


### Registration Form

In [35]:
import cv2
from insightface.app import FaceAnalysis

In [36]:
# configure face analysis
faceapp = FaceAnalysis(name='buffalo_sc',
                       root='insightface_model',
                       providers=['CPUExecutionProvider'])

faceapp.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)
# warning: don't set det_thresh < 0.3

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


## Step-1: Collect Person and Role

In [37]:
# python input
person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
    role = input("""
    Please choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
    """)
    
    if role in ('1','2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        
        break
    else:
        print('Invalid entry try again')
        if i == 3:
            print('Exceeds maximum trails')
            
key = person_name +'@'+ role
print('Your name = ', person_name)
print('Your role =', role)
print('Key =',key)

Enter your name:  Deepika

    Please choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
     1


Your name =  Deepika
Your role = Student
Key = Deepika@Student


##### Step-2: Collect Samples

In [38]:
cap = cv2.VideoCapture(0) # 0 default camera, 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    
    # get results from insightface model
    results = faceapp.get(frame,max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1),(x2,y2),(0,255,0),1)
        
        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
        
    if sample >= 200: 
        break
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'): # this is trigger only when I press letter q (lowercase q) in my keyboard
        break
        
cap.release()
cv2.destroyAllWindows()

In [39]:
len(face_embeddings)

200

##### Step-3: Optimize Data & Save in Redis Database

In [40]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [41]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [42]:
# save key and facial bytes into redis database
r.hset(name='academy:register',key=key,value=x_mean_bytes)

1